#  Earth Engine CHAI covariate extraction script

This notebook contains code to extract certain covariates from Earth Engine for CHAI modelling work by/for Kate Battle.

This is because some of her models require data at a higher resolution than the high-quality gapfilled data we already hold from sources such as MODIS. 

The data have been requested at 100m resolution where possible and this means using a data source such as Landsat. 

The code here only extracts the data from Earth Engine - it doesn't perform any enhancements such as gapfilling.

### Setup

To run this code you need to have the Earth Engine Python API installed, configured, authenticated, and available to the Python interpreter you're using as a notebook server. 

First you need a google account that's whitelisted to use Earth Engine. This is straightforward - just send a request using the links on the Earth Engine homepage (and your own Google account).

Next, instructions for installing and authenticating the API  manually are here:
https://developers.google.com/earth-engine/python_install_manual. Also see here for some further instructions on authorisation: https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/authorize_notebook_server.ipynb

Or you can use a hosted or a local Docker deployment with everything set up. Instructions here: https://developers.google.com/earth-engine/python_install



#### Load and test the API

In [232]:
# Attempt to load the EE API
import ee
ee.Initialize()

In [2]:
# Test the API setup - if this doesn't generate an error then you're good to go
image = ee.Image('srtm90_v4')
print(image.getInfo())

#### Open the Earth Engine datasets we'll be using

In [235]:
# Landsat 8 - only available from 2014 onwards
ls8_toa = ee.ImageCollection("LANDSAT/LC8_L1T_TOA")
ls8_raw = ee.ImageCollection("LANDSAT/LC8_L1T")
# Emissivity and modis landcover are used in the estimation of LST from brightness temp
asterEmis = ee.Image("NASA/ASTER_GED/AG100_003")
modis_landcover = ee.ImageCollection("MODIS/051/MCD12Q1")
# CHIRPS rainfall
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD")
# Hydrosheds DTM is gapfilled and corrected relative to the raw SRTM
elev = ee.Image("WWF/HydroSHEDS/03VFDEM")
# Not the best set of national boundaries but the only one I can find already in EE and ok for a bbox
nationalBoundaries = ee.FeatureCollection("USDOS/LSIB/2013")

#### Configure what we are going to extract
Landsat 8 is available from 2014.

For some countries Kate wants a synoptic file over the period of interest, others she wants one file per year.

Here we've only implemented a national bounding box as the unit to extract by, we could equally do subnational given a source of bbox coordinates.

We're exporting at 100m resolution and to a cloud storage bucket setup specifically for our group - change these values if required (e.g. if you don't have access to that bucket)

In [256]:
# Specify the years from which to harvest landsat 8 data, all the data from these years will be used 
# to create the seasonal composites which are then averaged
yrStart = 2016;
yrEnd = 2017;

# Specify whether to output a file for each year in the range or one averaged file
summariseToSynoptic = False;

# Specify the country to export data for as an ISO3 code. The export will cover the bounding box 
# of this country with a 10km buffer to allow for data discrepancies.
ISO3_TO_EXPORT = "MOZ";

# Export resolution in metres (so, in WGS84 the output resolution will vary with latitude)
EXPORT_RES_M = 100;
# Use a different cloud storage bucket if you need to.
CLOUDBUCKET = "ee-oxford-upload";

This next cell configures how the Earth Engine Landsat compositor algorithm runs - this attempts to make a cloud-free image for a given period of interest by ranking all the available images in that period based on a "cloud score" and then picking (at each pixel) the best value.

(This means that every pixel of the output comes from a different input image, and time)

In [236]:
# PROBABLY DON'T NEED TO CHANGE ANYTHING IN THIS CELL

# Specify the parameters for the ee.Algorithms.Landsat.simpleComposite algorithm. 
# You can add the "Seasonal best" map to the map viewer to see how well it's done, and tune these
# to suit. 
LS_CLOUD_SCORE_RANGE = 4;
LS_CLOUD_PERCENTILE = 5;

TASKLIST = []

### Code setup
Run all the following cells until the next note, to define the required functions

In [135]:
# Get a geometry for the area of interest 
geoms = nationalBoundaries.filterMetadata("iso_alpha3", "equals", ISO3_TO_EXPORT);
bufferedboxes = geoms.map(lambda f: (f.bounds().buffer(10000).bounds()))
  
combinedboxes = bufferedboxes.union(ee.ErrorMargin(10)).first()
#https://groups.google.com/d/msg/google-earth-engine-developers/TViMuO3ObeM/cpNNg-eMDAAJ
combinedboxesCoordsBizarrelyRequired = combinedboxes.getInfo()['geometry']['coordinates']

In [149]:
# A function to compute Enhanced Vegetation Index (EVI)  on Landsat 8 imagery
def computeEVI(image): 
    # The constants used to define EVI
    C1 = 6.0;
    C2 = 7.5;
    L = 1.0;
    G = 2.5;

    # Select the three bands we need
    red = image.select('B4');
    nir = image.select('B5');
    blue = image.select('B2');

    # Compute EVI
    evi = (nir.subtract(red)
        .divide(nir.add(red.multiply(C1)).subtract(blue.multiply(C2)).add(L))
        .multiply(G)
        .clamp(0.0, 1.0))

    # Rename the resulting band
    return (evi.rename(['evi'])
    .set('system:time_start', image.get('system:time_start')));


In [150]:
# A function to compute Tasseled Cap Brightness (TCB) on Landsat 8 imagery

# Coefficients from:
# Baig, M. H. A., et al. (2014). "Derivation of a tasselled cap transformation 
# based on Landsat 8 at-satellite reflectance." Remote Sensing Letters 5(5): 423-431.
def computeTCB(image):

    # The constants used to define EVI
    coef1 = 0.3029; # coef for blue (band 2)
    coef2 = 0.2786; # coef for green (band 3)
    coef3 = 0.4733; # coef for red (band 4)
    coef4 = 0.5599; # coef for NIR (band 5)
    coef5 = 0.5080; # coef for SWIR_1 (band 6)
    coef6 = 0.1872; # coef for SWIR_2 (band 7)


    # Select the bands we need
    blue = image.select('B2');
    green = image.select('B3');
    red = image.select('B4');
    nir = image.select('B5');
    swir1 = image.select('B6');
    swir2 = image.select('B7');

    tcb = (blue.multiply(coef1)
    .add(green.multiply(coef2))
    .add(red.multiply(coef3))
    .add(nir.multiply(coef4))
    .add(swir1.multiply(coef5))
    .add(swir2.multiply(coef6)));

    # Rename the resulting band
    return (tcb.rename(['tcb'])
    .set('system:time_start', image.get('system:time_start')));


In [151]:
# A function to compute Tasseled Cap Greenness (TCG) on Landsat 8 imagery
#
# Coefficients from:
# Baig, M. H. A., et al. (2014). "Derivation of a tasselled cap transformation 
# based on Landsat 8 at-satellite reflectance." Remote Sensing Letters 5(5): 423-431.
def computeTCG(image):
    # The constants used to define TCG
    coef1 = -0.2941; # coef for blue (band 2)
    coef2 = -0.243; # coef for green (band 3)
    coef3 = -0.5424; # coef for red (band 4)
    coef4 = 0.7276; # coef for NIR (band 5)
    coef5 = 0.0713; # coef for SWIR_1 (band 6)
    coef6 = -0.1608; # coef for SWIR_2 (band 7)

    # Select the bands we need
    blue = image.select('B2');
    green = image.select('B3');
    red = image.select('B4');
    nir = image.select('B5');
    swir1 = image.select('B6');
    swir2 = image.select('B7');

    tcg = (blue.multiply(coef1)
    .add(green.multiply(coef2))
    .add(red.multiply(coef3))
    .add(nir.multiply(coef4))
    .add(swir1.multiply(coef5))
    .add(swir2.multiply(coef6)));

    # Rename the resulting band
    return (tcg.rename(['tcg'])
    .set('system:time_start', image.get('system:time_start')));


In [152]:
# A function to compute Tasseled Cap Wetness (TCW) on Landsat 8 imagery
#
# Coefficients from:
# Baig, M. H. A., et al. (2014). "Derivation of a tasselled cap transformation 
# based on Landsat 8 at-satellite reflectance." Remote Sensing Letters 5(5): 423-431.
def computeTCW(image):
    # The constants used to define EVI
    coef1 = 0.1511; # coef for blue (band 2)
    coef2 = 0.1973; # coef for green (band 3)
    coef3 = 0.3283; # coef for red (band 4)
    coef4 = 0.3407; # coef for NIR (band 5)
    coef5 = -0.7117; # coef for SWIR_1 (band 6)
    coef6 = -0.4559; # coef for SWIR_2 (band 7)


    # Select the bands we need
    blue = image.select('B2');
    green = image.select('B3');
    red = image.select('B4');
    nir = image.select('B5');
    swir1 = image.select('B6');
    swir2 = image.select('B7');

    tcw = (blue.multiply(coef1)
    .add(green.multiply(coef2))
    .add(red.multiply(coef3))
    .add(nir.multiply(coef4))
    .add(swir1.multiply(coef5))
    .add(swir2.multiply(coef6)));

    # Rename the resulting band
    return (tcw.rename(['tcw'])
    .set('system:time_start', image.get('system:time_start')));


In [154]:
# Map MODIS MCD12Q1 landcover to emissivity for use in calculating land surface temperature 
# from landsat brightness temperatures
# Uses the NPP landcover classification layer as described at:
# https:#lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table/mcd12q1
# and emissivity values estimated from these by sticking a finger into the air in the 
# vicinity of https:#fromgistors.blogspot.com/2016/09/estimation-of-land-surface-temperature.html
def NPPtoEmissivity(image):
    # NPP MODIS classification is 
    # 0 - water
    # 1-5 - types of forest
    # 6 - grass
    # 7 - non-vegetated
    # 8 - urban
    lcClasses =   [0,1,2,3,4,5,6,7,8,254,255];
    
    # emissivity numbers are a matter of guesstimation. Using values from:
    # https:#fromgistors.blogspot.com/2016/09/estimation-of-land-surface-temperature.html
    # which gives
    # water 0.98
    # Built-up	0.94
    # Vegetation	0.98
    # Bare soil	0.93
    
    # multiply by 100 as the remap function needs ints
    emisClasses = [98, 98, 98, 98, 98, 98, 98, 93, 94, 25400, 25500];
    # Note that "remap" is the EE equivalent of "reclass". "Land_Cover_Type_4" is the name 
    # of the NPP band in the MODIS image
    emisMap = image.remap(lcClasses, emisClasses, None, 'Land_Cover_Type_4');
    return emisMap.divide(100.0);


In [44]:
# Estimate Land Surface Temperature from (separately) bands 10 and 11 of the landsat image 
# (the brightness temperature). Uses the most recent modis landcover image (2013) and some 
# highly official coefficients to estimate emissivity.
# NB the two bands give rather different values. Nobody knows which is less wrong.
def brightnessToLst(image):
    emis2013 = (modis_landcover
        .filter(ee.Filter.calendarRange(2013,2013,"year"))
        .map(NPPtoEmissivity)
        .first());
    emis2013 = ee.Image(emis2013);  
    lst10 = (image.expression
             (
                'BT / (1 + (lambda * BT / plnk) * log(emis))', 
                {
                  'BT': image.select('B10'),
                  'lambda' : 10.8e-6,
                  'plnk': 1.4388e-2,
                  'emis': emis2013.select('remapped')
                }
            )
        .select('B10')
        .rename(['lst_band10'])
    );
    lst11 = (image.expression
        (
            'BT / (1 + (lambda * BT / plnk) * log(emis))', 
            {
              'BT': image.select('B11'),
              'lambda' : 12e-6,
              'plnk': 1.4388e-2,
              'emis': emis2013.select('remapped')
            }
        )
        .select('B11')
        .rename(['lst_band11'])
    );
    return image.addBands(lst10).addBands(lst11);


In [45]:
# Function to calculate evi, tcb, tcg, tcw, lst10, lst11 for a single landsat 8 image
# Returns the input image with each of these indices added as a band named
# "evi", "tcb", "tcg", "tcw", "lst_band10", and "lst_band11".
# This uses the functions defined separately for each individual index, and is 
# provided as a convenience wrapper
def calcIndices(img):
    # Calculate NDVI as a band for optional sorting by that
    ndvi = img.normalizedDifference(['B5', 'B4']).select([0], ['ndvi']);
    # Calculate EVI using the function above
    evi = computeEVI(img);
    # Calculate the Tasseled Cap Bands using the functions above
    tcb = computeTCB(img);
    # tcbINV = ee.Image(-1).multiply(tcb).select([0], ['tcbINV']);
    tcg = computeTCG(img);
    tcw = computeTCW(img);

    # Add the bands to the image
    img = img.addBands(ndvi);
    img = img.addBands(evi);
    img = img.addBands(tcb);
    # img = img.addBands(tcbINV);
    img = img.addBands(tcg);
    img = img.addBands(tcw);
    # the temperature function adds the bands to the image rather than returning the bands alone
    img = brightnessToLst(img);

    return img;


No matter whether we are outputting an image representing one or several years of data, the image will be calculated at the average of four seasonal subset images. Each of those seasonal images will be calculated from all the data for that season occurring in the year(s) of interest. This is to minimise the effect of seasonally-biased cloud cover meaning we could get more input images at a particular time of year (when, say, the ground may be less green).

In [144]:
# Function to calculate a seasonally-balanced annual mean image from the landsat 8 image stack.
# The returned image is comprised of a mean of 4 seasonal (spring, summer, autumn, winter) images,
# where each of those 4 images is generated using the simple cloud-avoiding compositor function for 
# all source images occurring in the specified years in the relevant season. The image has the evi, ndvi, 
# etc bands added.

def landsatSeasonalSummary(yrFrom, yrTo):
    # get the relevant year(s) data from the imagecollection
    yearData = ls8_raw.filter(ee.Filter.calendarRange(yrFrom,yrTo,"year"));
    # filter the input data into 4 seasons
    spring = yearData.filter(ee.Filter.calendarRange(4,6,"month"));
    summer = yearData.filter(ee.Filter.calendarRange(7,9,"month"));
    autumn = yearData.filter(ee.Filter.calendarRange(10,12,"month"));
    winter = yearData.filter(ee.Filter.calendarRange(1,3,"month"));

    # calculate the indices for each season separately, using the built-in simple composite 
    # function to select the best cloud free value at each pixel from the available images
    springBest = calcIndices(
    ee.Algorithms.Landsat.simpleComposite(**{
      'collection': spring,
      'percentile': LS_CLOUD_PERCENTILE,
      'cloudScoreRange': LS_CLOUD_SCORE_RANGE,
      'asFloat':True,
      'maxDepth':200
    })
    );
    summerBest = calcIndices(
    ee.Algorithms.Landsat.simpleComposite(**{
      'collection': summer,
      'percentile': LS_CLOUD_PERCENTILE,
      'cloudScoreRange': LS_CLOUD_SCORE_RANGE,
      'asFloat':True,
      'maxDepth':200
    })
    );
    autumnBest = calcIndices(
    ee.Algorithms.Landsat.simpleComposite(**{
      'collection': autumn,
      'percentile': LS_CLOUD_PERCENTILE,
      'cloudScoreRange': LS_CLOUD_SCORE_RANGE,
      'asFloat':True,
      'maxDepth':200
    })
    );
    winterBest = calcIndices(
    ee.Algorithms.Landsat.simpleComposite(**{
      'collection': winter,
      'percentile': LS_CLOUD_PERCENTILE,
      'cloudScoreRange': LS_CLOUD_SCORE_RANGE,
      'asFloat':True,
      'maxDepth':200
    })
    );

    # calculate a "seasonal" or balanced mean from the 4 seasonal images
    # THIS IS OUR OUTPUT IMAGE - we just select the bands out from it
    seasonalBest = (ee.ImageCollection
    .fromImages([springBest,summerBest,autumnBest,winterBest])
    .mean());
    return seasonalBest;


In [127]:
# just a bit of sugar for calling the export to cloud storage routine
def xFunc(desc, fnPrefix, img):
    #task_config = {
    #    'description': desc,
    #    'scale': EXPORT_RES_M,  
    #    'region': combinedboxes,
    #    'bucket': CLOUDBUCKET,
    #    'maxPixels': 400000000,
    #    'fileNamePrefix': fnPrefix,
    #    'crs': 'EPSH:4326'
    #}

    #task = ee.batch.Export.image(img, desc, task_config)

    task = ee.batch.Export.image.toCloudStorage(**{
        'image': img,
        'description': desc,
        'bucket': CLOUDBUCKET,
        'fileNamePrefix': fnPrefix,
        'scale': EXPORT_RES_M,
        'maxPixels': 400000000,
        'region': combinedboxesCoordsBizarrelyRequired,
        'crs': 'EPSG:4326'
    });
    return task

In [189]:
def runCHAIExportsForYr(yrFrom, yrTo):
    # create an output image representing the mean of the four seasonal images which are 
    # themselves drawn from that season's data in all the input years
    seasonalBest = landsatSeasonalSummary(yrFrom, yrTo);
    # for the output filename
    yrPhrase = str(yrFrom) if yrFrom == yrTo else str(yrFrom) + "-" + str(yrTo);
    
    # all the landsat reflectance ones:
    lsIndices = ['tcb', 'tcw', 'tcg', 'evi', 'ndvi'];
    for idtag in lsIndices:
        fnPrefix = (ISO3_TO_EXPORT + "_" + idtag + "_ls8_" +
          str(EXPORT_RES_M) + "m_" + yrPhrase + "_seasonal_mean");
        desc = ISO3_TO_EXPORT + "-" + idtag + "-" + yrPhrase + "-exp";
        tmpImg = seasonalBest.select(idtag);
        t = xFunc(desc, fnPrefix, tmpImg);
        TASKLIST.append(t)
    
    # the temp ones need slight modification to output in celsius
    tmpImg = seasonalBest.select('lst_band10').subtract(273.15);
    fnPrefix = (ISO3_TO_EXPORT + '_lsTemp10_ls8_' + str(EXPORT_RES_M)
                + "m_" + yrPhrase + '_seasonal_mean');
    desc = ISO3_TO_EXPORT + "-" + 'lsTemp10' + "-" + yrPhrase + "-exp";
    t = xFunc(desc, fnPrefix, tmpImg);
    TASKLIST.append(t)
    
    tmpImg = seasonalBest.select('lst_band11').subtract(273.15);
    fnPrefix = (ISO3_TO_EXPORT + '_lsTemp11_ls8_' + str(EXPORT_RES_M) 
                + "m_" + yrPhrase + '_seasonal_mean');
    desc = ISO3_TO_EXPORT + "-" + 'lsTemp11' + "-" + yrPhrase + "-exp";
    t = xFunc(desc, fnPrefix, tmpImg);
    TASKLIST.append(t)

    # we aren't using brightness temps
    #tmpImg = seasonalBest.select('B10').subtract(273.15);
    #fnPrefix = (ISO3_TO_EXPORT + '_bTemp10_ls8_' + EXPORT_RES_M 
    #            + "m_" + yrPhrase + '_seasonal_mean');
    #desc = ISO3_TO_EXPORT + "-" + 'bTemp10' + "-" + yrPhrase + "-exp";
    #xFunc(desc, fnPrefix, tmpImg);
    
    #tmpImg = seasonalBest.select('B11').subtract(273.15);
    #fnPrefix = (ISO3_TO_EXPORT + '_bTemp11_ls8_' + EXPORT_RES_M 
    #            + "m_" + yrPhrase + '_seasonal_mean');
    #desc = ISO3_TO_EXPORT + "-" + 'bTemp11' + "-" + yrPhrase + "-exp";
    #xFunc(desc, fnPrefix, tmpImg);

    
    # Summarise the CHIRPS rainfall data: we need to do annual SUM with this not annual MEAN
    chirpsImgs = [];
    for y in range (yrFrom, yrTo+1):
        chirpsYr = chirps.filter(ee.Filter.calendarRange(y, y, "year")).sum();
        chirpsImgs.append(chirpsYr);
    
    chirpsMean = ee.ImageCollection.fromImages(chirpsImgs).mean();
    # Map.addLayer(chirps2014,{palette:"FF0000,0000FF",min:0,max:3000},'2014');
    fnPrefix = (ISO3_TO_EXPORT + '_chirps_' + str(EXPORT_RES_M) 
                + 'm_' + yrPhrase + '_annual_average');
    desc = ISO3_TO_EXPORT + "-" + 'chirps' + "-" + yrPhrase + "-exp";
    t = xFunc(desc, fnPrefix, chirpsMean);
    TASKLIST.append(t)


# Initialise and run the exports!
This next cell is the one that will actually build the export requests for each covariate and year. The requests are accumulated as Task objects which are queued but not started yet. Run this cell.

In [257]:
TASKLIST = []
# Elevation we just export as-is (barring the resolution upsampling)
#print(elev.projection());
fnPrefix = ISO3_TO_EXPORT + '_hydrosheds_elevation_' + str(EXPORT_RES_M) + 'm';
desc = ISO3_TO_EXPORT + '-' + 'elev-exp';
t = xFunc(desc, fnPrefix, elev);
TASKLIST.append(t)

if (summariseToSynoptic):
    runCHAIExportsForYr(yrStart, yrEnd);

else:
    for y in range (yrStart, yrEnd+1):
        runCHAIExportsForYr(y, y);
  

Now just check that they look like what you were expecting

In [258]:
TASKLIST

[<Task EXPORT_IMAGE: MOZ-elev-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-tcb-2016-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-tcw-2016-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-tcg-2016-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-evi-2016-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-ndvi-2016-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-lsTemp10-2016-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-lsTemp11-2016-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-chirps-2016-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-tcb-2017-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-tcw-2017-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-tcg-2017-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-evi-2017-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-ndvi-2017-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-lsTemp10-2017-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-lsTemp11-2017-exp (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: MOZ-chirps-2017-exp (UNSUBMITTED)>]

Start all of the tasks (or of course, remove some if you don't want to)

In [261]:
for t in TASKLIST:
    print(t.config['description'])
    t.start()

MOZ-elev-exp
MOZ-tcb-2016-exp
MOZ-tcw-2016-exp
MOZ-tcg-2016-exp
MOZ-evi-2016-exp
MOZ-ndvi-2016-exp
MOZ-lsTemp10-2016-exp
MOZ-lsTemp11-2016-exp
MOZ-chirps-2016-exp
MOZ-tcb-2017-exp
MOZ-tcw-2017-exp
MOZ-tcg-2017-exp
MOZ-evi-2017-exp
MOZ-ndvi-2017-exp
MOZ-lsTemp10-2017-exp
MOZ-lsTemp11-2017-exp
MOZ-chirps-2017-exp


The tasks will now be running on the EE servers. Not necessarily all at once - how they're actually scheduled is out of our hands now. But you can check their status on the task objects:

In [ ]:
[t.status() for t in TASKLIST]

or re-retrieve them from the server - which will also show any other older tasks that are still recorded not just those that have just been created

In [270]:
ts = ee.batch.Task.list()

In [271]:
ts

[<Task EXPORT_IMAGE: MOZ-chirps-2017-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: MOZ-lsTemp11-2017-exp (FAILED)>,
 <Task EXPORT_IMAGE: MOZ-lsTemp10-2017-exp (FAILED)>,
 <Task EXPORT_IMAGE: MOZ-ndvi-2017-exp (FAILED)>,
 <Task EXPORT_IMAGE: MOZ-evi-2017-exp (FAILED)>,
 <Task EXPORT_IMAGE: MOZ-tcg-2017-exp (FAILED)>,
 <Task EXPORT_IMAGE: MOZ-tcw-2017-exp (FAILED)>,
 <Task EXPORT_IMAGE: MOZ-tcb-2017-exp (FAILED)>,
 <Task EXPORT_IMAGE: MOZ-chirps-2016-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: MOZ-lsTemp11-2016-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: MOZ-lsTemp10-2016-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: MOZ-ndvi-2016-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: MOZ-evi-2016-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: MOZ-tcg-2016-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: MOZ-tcw-2016-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: MOZ-tcb-2016-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: MOZ-elev-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: GTM-chirps-2014-2016-exp (COMPLETED)>,
 <Task EXPORT_IMAGE: GTM-lsTemp11-2014-2016-exp (COMP

When the tasks have completed the files should be present in the specified cloud storage bucket. You can now download them directly from the web interface at https://console.cloud.google.com/storage/browser/BUCKET-NAME/ 

Or you can use the gsutil command line tools (much more convenient if there's lots). e.g.

`gsutil -m mv gs://BUCKET-NAME/ZWE*.tif E:\My\Local\Folder\Path`

Don't forget to delete them from the bucket after downloading (if not using mv) to avoid possible storage charges.

If you need to cancel a task after start() has been called on it (whether it's running yet or not) then just do this:

In [251]:
[t.cancel() for t in ts if t.active()]

[]